In [29]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [30]:
df = pd.read_csv('indicadoresEquipoHistoricoModelo.csv',sep=",")
df

,id_indicadores_equipo_prepartido,id_partido,porcentaje local ganados en sitio,porcentaje local ganados en general,porcentaje local empatados en sitio,porcentaje local empatados en general,porcentaje local perdidos en sitio,porcentaje local perdidos en general,porcentaje visitante ganados en sitio,porcentaje visitante ganados en general,...,proporcion visitante cambios alineacion centrocampista en sitio,proporcion visitante cambios alineacion centrocampista en general,proporcion local cambios alineacion delantero sitio,proporcion local cambios alineacion delantero en general,proporcion visitante cambios alineacion delantero en sitio,proporcion visitante cambios alineacion delantero en general,resultado_local,resultado_visitante,resultado_partido,jornada
0,89.0,98.0,0.00,33.33,25.00,11.11,75.00,55.56,0.00,11.11,...,1.00,0.89,1.75,2.22,2.00,1.56,2,0,1,10
1,90.0,96.0,60.00,44.44,40.00,33.33,0.00,22.22,20.00,11.11,...,1.40,1.56,1.20,1.56,0.80,0.89,1,0,1,10
2,91.0,94.0,50.00,22.22,50.00,44.44,0.00,33.33,20.00,33.33,...,1.60,1.33,0.75,0.89,1.80,1.56,1,1,X,10
3,92.0,93.0,0.00,11.11,40.00,33.33,60.00,55.56,50.00,75.00,...,2.25,1.88,1.20,1.11,1.50,1.75,0,3,2,10
4,93.0,97.0,40.00,25.00,20.00,25.00,40.00,50.00,80.00,88.89,...,1.80,2.11,1.60,1.88,0.80,0.78,1,1,X,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,345.0,341.0,88.24,61.76,5.88,11.76,5.88,26.47,17.65,23.53,...,0.88,0.88,1.41,1.21,0.88,0.91,1,0,1,35
256,346.0,346.0,47.06,38.24,41.18,38.24,11.76,23.53,11.76,5.88,...,1.41,1.24,1.12,1.26,1.12,1.09,3,2,1,35
257,347.0,348.0,47.06,38.24,29.41,23.53,23.53,38.24,23.53,20.59,...,1.12,0.97,0.88,0.91,1.00,1.06,0,0,X,35
258,348.0,342.0,23.53,11.76,47.06,41.18,29.41,47.06,11.76,29.41,...,0.82,0.74,0.94,1.32,1.00,0.85,1,0,1,35


In [31]:
# Eliminar la columna id_partido
df.drop(['id_partido','jornada','id_indicadores_equipo_prepartido'], axis=1, inplace=True)

In [32]:
# Separar las características (porcentajes) y la clase a predecir (resultado_partido)
X = df.drop(['resultado_partido','resultado_local','resultado_visitante'], axis=1)

In [33]:
y_resultado=df['resultado_partido']
y_resultado_local=df['resultado_local']
y_resultado_visitante=df['resultado_visitante']

In [35]:
# Codificar las etiquetas (resultado_partido)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_resultado)

# Convertir las etiquetas a one-hot encoding
y_categorical = to_categorical(y_encoded)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [37]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Capa de salida con 3 neuronas para las 3 clases
])

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

Epoch 1/50
12/12 [==============================] - 1s 17ms/step - loss: 7.2772 - accuracy: 0.4332 - val_loss: 9.8383 - val_accuracy: 0.2381
Epoch 2/50
12/12 [==============================] - 0s 4ms/step - loss: 7.2326 - accuracy: 0.3797 - val_loss: 2.8354 - val_accuracy: 0.5714
Epoch 3/50
12/12 [==============================] - 0s 4ms/step - loss: 3.5539 - accuracy: 0.4225 - val_loss: 2.8990 - val_accuracy: 0.6667
Epoch 4/50
12/12 [==============================] - 0s 4ms/step - loss: 3.1055 - accuracy: 0.4706 - val_loss: 2.2685 - val_accuracy: 0.5714
Epoch 5/50
12/12 [==============================] - 0s 5ms/step - loss: 2.6439 - accuracy: 0.4759 - val_loss: 2.7568 - val_accuracy: 0.1905
Epoch 6/50
12/12 [==============================] - 0s 3ms/step - loss: 2.8911 - accuracy: 0.4973 - val_loss: 1.8511 - val_accuracy: 0.4762
Epoch 7/50
12/12 [==============================] - 0s 4ms/step - loss: 1.6901 - accuracy: 0.5401 - val_loss: 1.8211 - val_accuracy: 0.4762
Epoch 8/50
12/12 [=

In [40]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

2/2 [==============================] - 0s 23ms/step - loss: 1.5405 - accuracy: 0.5192
Pérdida en el conjunto de prueba: 1.5405
Precisión en el conjunto de prueba: 0.5192


In [41]:
# Predecir las probabilidades de las clases en el conjunto de prueba
y_pred_proba = model.predict(X_test)

# Decodificar las etiquetas predichas y reales
y_test_decoded = label_encoder.inverse_transform(np.argmax(y_test, axis=1))
y_pred_decoded = label_encoder.inverse_transform(np.argmax(y_pred_proba, axis=1))

# Mostrar las probabilidades para cada clase junto con la clase real
for i, (probs, true_class, pred_class) in enumerate(zip(y_pred_proba, y_test_decoded, y_pred_decoded)):
    print(f"Partido {i+1}:")
    print(f"  Clase real: {true_class}")
    print(f"  Clase predicha: {pred_class}")
    for class_name, prob in zip(label_encoder.classes_, probs):
        print(f"  Probabilidad de {class_name}: {prob:.4f}")

2/2 [==============================] - 0s 6ms/step
Partido 1:
  Clase real: 1
  Clase predicha: 1
  Probabilidad de 1: 0.7251
  Probabilidad de 2: 0.0180
  Probabilidad de X: 0.2569
Partido 2:
  Clase real: 1
  Clase predicha: 1
  Probabilidad de 1: 0.8887
  Probabilidad de 2: 0.0005
  Probabilidad de X: 0.1108
Partido 3:
  Clase real: 1
  Clase predicha: 1
  Probabilidad de 1: 0.9999
  Probabilidad de 2: 0.0000
  Probabilidad de X: 0.0001
Partido 4:
  Clase real: 2
  Clase predicha: 1
  Probabilidad de 1: 0.5772
  Probabilidad de 2: 0.4109
  Probabilidad de X: 0.0118
Partido 5:
  Clase real: 1
  Clase predicha: 1
  Probabilidad de 1: 0.6427
  Probabilidad de 2: 0.0731
  Probabilidad de X: 0.2842
Partido 6:
  Clase real: 1
  Clase predicha: X
  Probabilidad de 1: 0.0728
  Probabilidad de 2: 0.0081
  Probabilidad de X: 0.9191
Partido 7:
  Clase real: 1
  Clase predicha: 2
  Probabilidad de 1: 0.0448
  Probabilidad de 2: 0.9218
  Probabilidad de X: 0.0334
Partido 8:
  Clase real: 1
  Cla